# Recommender Systems Tutorial
In this tutorial, we'll provide a simple walkthrough of how to use Snorkel to build a recommender system.
We consider a setting similar to the [Netflix challenge](https://www.kaggle.com/netflix-inc/netflix-prize-data), but with books instead of movies.
We have a set of users and books, and for each user we know the set of books they have interacted with (read or marked as to-read).
We don't have the user's numerical ratings for the books they read, except in a small number of cases.
We also have some text reviews written by users.

Our goal is to build a recommender system by training a classifier to predict whether a user will read and like any given book.
We'll train our model over a user-book pair to predict a `rating` (a `rating` of 1 means the user will read and like the book).
To simplify inference, we'll represent a user by the set of books they interacted with (rather than learning a specific representation for each user).
Once we have this model trained, we can use it to recommend books to a user when they visit the site.
For example, we can just predict the rating for the user paired with a book for a few thousand likely books, then pick the books with the top ten predicted ratings.

Of course, there are many other ways to approach this problem.
The field of [recommender systems](https://en.wikipedia.org/wiki/Recommender_system) is a very well studied area with a wide variety of settings and approaches, and we just focus on one of them.

We will use the [Goodreads](https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home) dataset, from
"Item Recommendation on Monotonic Behavior Chains", RecSys'18 (Mengting Wan, Julian McAuley), and "Fine-Grained Spoiler Detection from Large-Scale Review Corpora", ACL'19 (Mengting Wan, Rishabh Misra, Ndapa Nakashole, Julian McAuley).
In this dataset, we have user interactions and reviews for Young Adult novels from the Goodreads website, along with metadata (like `title` and `authors`) for the novels.

In [1]:
import logging
import os

logging.basicConfig(level=logging.INFO)


if os.path.basename(os.getcwd()) == "snorkel-tutorials":
    os.chdir("recsys")

## Loading Data

We start by running the `download_and_process_data` function.
The function returns the `df_train`, `df_test`, `df_dev`, `df_valid` dataframes, which correspond to our training, test, development, and validation sets.
Each of those dataframes has the following fields:
* `user_idx`: A unique identifier for a user.
* `book_idx`: A unique identifier for a book that is being rated by the user.
* `book_idxs`: The set of books that the user has interacted with (read or planned to read).
* `review_text`: Optional text review written by the user for the book.
* `rating`: Either `0` (which means the user did not read or did not like the book) or `1` (which means the user read and liked the book). The `rating` field is missing for `df_train`.
Our objective is to predict whether a given user (represented by the set of book_idxs the user has interacted with) will read and like any given book.
That is, we want to train a model that takes a set of `book_idxs` (the user) and a single `book_idx` (the book to rate) and predicts the `rating`.

In addition, `download_and_process_data` also returns the `df_books` dataframe, which contains one row per book, along with metadata for that book (such as `title` and `first_author`).

In [2]:
from utils import download_and_process_data

(df_train, df_test, df_dev, df_valid), df_books = download_and_process_data()

df_books.head()

/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/snorkel-t

/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

I0813 19:08:42.916476 139779728627520 utils.py:219] Processing book data


I0813 19:08:57.738869 139779728627520 utils.py:221] Processing interaction data


/home/ubuntu/snorkel-tutorials/recsys/utils.py:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_interactions_nz["rating_4_5"] = df_interactions_nz.rating.map(ratings_map)
I0813 19:10:44.231085 139779728627520 utils.py:226] Processing review data


I0813 19:11:23.902697 139779728627520 utils.py:228] Joining interaction data


,authors,average_rating,book_id,country_code,description,is_ebook,language_code,ratings_count,similar_books,text_reviews_count,title,first_author,book_idx
3,[293603],4.35,10099492,US,It all comes down to this.\nVlad's running out...,True,eng,152,"[25861113, 7430195, 18765937, 6120544, 3247550...",9,Twelfth Grade Kills (The Chronicles of Vladimi...,293603,0
4,[4018722],3.71,22642971,US,The future world is at peace.\nElla Shepherd h...,True,eng,1525,"[20499652, 17934493, 13518102, 16210411, 17149...",428,The Body Electric,4018722,1
5,[6537142],3.89,31556136,US,A gorgeously written and deeply felt literary ...,True,,109,[],45,Like Water,6537142,2
12,"[6455200, 5227552]",3.90,18522274,US,Zoe Vanderveen is on the run with her captor t...,True,en-US,191,"[25063023, 18553080, 17567752, 18126509, 17997...",6,"Volition (The Perception Trilogy, #2)",6455200,3
13,[187837],3.19,17262776,US,"The war is over, but for thirteen-year-old Rac...",True,eng,248,"[16153997, 10836616, 17262238, 16074827, 13628...",68,Little Red Lies,187837,4


We look at a sample of the labeled development set.
As an example, we want our final recommendations model to be able to predict that a user who has interacted with `book_idxs` (25743, 22318, 7662, 6857, 83, 14495, 30664, ...) would either not read or not like the book with `book_idx` 22764 (first row), while a user who has interacted with `book_idxs` (3880, 18078, 9092, 29933, 1511, 8560, ...) would read and like the book with `book_idx` 3181 (second row).

In [3]:
df_dev.sample(frac=1, random_state=12).head()

,user_idx,book_idxs,book_idx,rating,review_text
658649,25509,"(15453, 15065, 15651, 21312, 24778, 11093, 878...",23921,1,"Wow. Final super inesperado, me dejo de piedra..."
526819,20342,"(25234, 23735, 11313, 16477, 3074, 9898, 3645,...",4352,1,NaN
693666,26965,"(21658, 3157, 14224, 6318, 19319, 13933, 21701...",21725,1,NaN
104242,4090,"(13518, 30807, 24868, 23799, 1120, 24107, 4503...",9045,1,NaN
260383,10074,"(22868, 21949, 11821, 29801, 30520, 28357, 123...",28046,1,NaN


## Writing Labeling Functions

In [4]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

If a user has interacted with several books written by an author, there is a good chance that the user will read and like other books by the same author.
We express this as a labeling function, using the `first_author` field in the `df_books` dataframe.
We picked the threshold 15 by plotting histograms and running error analysis using the dev set.

In [5]:
from snorkel.labeling.lf import labeling_function

book_to_first_author = dict(zip(df_books.book_idx, df_books.first_author))
first_author_to_books_df = df_books.groupby("first_author")[["book_idx"]].agg(set)
first_author_to_books = dict(
    zip(first_author_to_books_df.index, first_author_to_books_df.book_idx)
)


@labeling_function(
    resources=dict(
        book_to_first_author=book_to_first_author,
        first_author_to_books=first_author_to_books,
    )
)
def shared_first_author(x, book_to_first_author, first_author_to_books):
    author = book_to_first_author[x.book_idx]
    same_author_books = first_author_to_books[author]
    num_read = len(set(x.book_idxs).intersection(same_author_books))
    return POSITIVE if num_read > 15 else ABSTAIN

We can also leverage the long text reviews written by users to guess whether they liked or disliked a book.
For example, the third `df_dev` entry above has a review with the text `'4.5 STARS'`, which indicates that the user liked the book.
We write a simple LF that looks for similar phrases to guess the user's rating of a book.
We interpret >= 4 stars to indicate a positive rating, while < 4 stars is negative.

In [6]:
low_rating_strs = [
    "one star",
    "1 star",
    "two star",
    "2 star",
    "3 star",
    "three star",
    "3.5 star",
    "2.5 star",
    "1 out of 5",
    "2 out of 5",
    "3 out of 5",
]
high_rating_strs = ["5 stars", "five stars", "four stars", "4 stars", "4.5 stars"]


@labeling_function(
    resources=dict(low_rating_strs=low_rating_strs, high_rating_strs=high_rating_strs)
)
def stars_in_review(x, low_rating_strs, high_rating_strs):
    if not isinstance(x.review_text, str):
        return ABSTAIN
    for low_rating_str in low_rating_strs:
        if low_rating_str in x.review_text.lower():
            return NEGATIVE
    for high_rating_str in high_rating_strs:
        if high_rating_str in x.review_text.lower():
            return POSITIVE
    return ABSTAIN

We can also run [TextBlob](https://textblob.readthedocs.io/en/dev/index.html), a tool that provides a pretrained sentiment analyzer, on the reviews, and use its polarity and subjectivity scores to estimate the user's rating for the book.
As usual, these thresholds were picked by analyzing the score distributions and running error analysis.

In [7]:
from snorkel.preprocess import preprocessor
from textblob import TextBlob


@preprocessor(memoize=True)
def textblob_polarity(x):
    if isinstance(x.review_text, str):
        x.blob = TextBlob(str(x.review_text))
    else:
        x.blob = None
    return x


# Label high polarity reviews as positive.
@labeling_function(pre=[textblob_polarity])
def polarity_positive(x):
    if x.blob:
        if x.blob.polarity > 0.3:
            return POSITIVE
    return ABSTAIN


# Label high subjectivity reviews as positive.
@labeling_function(pre=[textblob_polarity])
def subjectivity_positive(x):
    if x.blob:
        if x.blob.subjectivity > 0.75:
            return POSITIVE
    return ABSTAIN


# Label low polarity reviews as negative.
@labeling_function(pre=[textblob_polarity])
def polarity_negative(x):
    if x.blob:
        if x.blob.polarity < 0.0:
            return NEGATIVE
    return ABSTAIN

In [8]:
from snorkel.labeling import PandasLFApplier, LFAnalysis

lfs = [
    stars_in_review,
    shared_first_author,
    polarity_positive,
    subjectivity_positive,
    polarity_negative,
]

applier = PandasLFApplier(lfs)
L_dev = applier.apply(df_dev)
LFAnalysis(L_dev, lfs).lf_summary(df_dev.rating)

  0%|          | 0/8310 [00:00<?, ?it/s]

  1%|          | 61/8310 [00:00<00:14, 586.97it/s]

  2%|▏         | 155/8310 [00:00<00:12, 661.32it/s]

  3%|▎         | 273/8310 [00:00<00:10, 756.62it/s]

  4%|▍         | 336/8310 [00:00<00:11, 705.60it/s]

...

 96%|█████████▋| 8000/8310 [00:08<00:00, 949.68it/s]

 98%|█████████▊| 8103/8310 [00:08<00:00, 962.73it/s]

 99%|█████████▉| 8257/8310 [00:08<00:00, 1084.09it/s]

100%|██████████| 8310/8310 [00:08<00:00, 991.34it/s] 

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
stars_in_review,0,"[0, 1]",0.024428,0.005415,0.002046,163,40,0.802956
shared_first_author,1,[1],0.034657,0.003971,0.001083,232,56,0.805556
polarity_positive,2,[1],0.055235,0.015644,0.000842,380,79,0.827887
subjectivity_positive,3,[1],0.019254,0.014801,0.003249,120,40,0.750000
polarity_negative,4,[0],0.022864,0.005535,0.004452,123,67,0.647368


### Applying labeling functions to the training set

We apply the labeling functions to the training set, and then filter out examples unlabeled by any LF to form our final training set.

In [9]:
from snorkel.labeling.model.label_model import LabelModel

L_train = applier.apply(df_train)
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=5000, seed=123, log_freq=20, lr=0.01)
Y_train_preds = label_model.predict(L_train)

  0%|          | 0/794516 [00:00<?, ?it/s]

  0%|          | 1/794516 [00:00<25:11:44,  8.76it/s]

  0%|          | 151/794516 [00:00<17:40:40, 12.48it/s]

  0%|          | 266/794516 [00:00<12:25:49, 17.75it/s]

  0%|          | 369/794516 [00:00<8:45:55, 25.17it/s] 

...

I0813 19:24:07.449849 139779728627520 logger.py:79] [4920 epochs]: TRAIN:[loss=0.000]


I0813 19:24:07.463464 139779728627520 logger.py:79] [4940 epochs]: TRAIN:[loss=0.000]


I0813 19:24:07.477016 139779728627520 logger.py:79] [4960 epochs]: TRAIN:[loss=0.000]


I0813 19:24:07.490839 139779728627520 logger.py:79] [4980 epochs]: TRAIN:[loss=0.000]


I0813 19:24:07.503840 139779728627520 label_model.py:806] Finished Training


In [10]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, Y_train_preds_filtered = filter_unlabeled_dataframe(
    df_train, Y_train_preds, L_train
)
df_train_filtered["rating"] = Y_train_preds_filtered

/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Rating Prediction Model
We write a Keras model for predicting ratings given a user's book list and a book (which is being rated).
The model represents the list of books the user interacted with, `books_idxs`, by learning an embedding for each idx, and averaging the embeddings in `book_idxs`.
It learns another embedding for the `book_idx`, the book to be rated.
Then it concatenates the two embeddings and uses an [MLP](https://en.wikipedia.org/wiki/Multilayer_perceptron) to compute the probability of the `rating` being 1.
This type of model is common in large-scale recommender systems, for example, the [YouTube recommender system](https://ai.google/research/pubs/pub45530).

In [11]:
import numpy as np
import tensorflow as tf
from utils import precision_batch, recall_batch, f1_batch

n_books = max([max(df.book_idx) for df in [df_train, df_test, df_dev, df_valid]])


# Keras model to predict rating given book_idxs and book_idx.
def get_model(embed_dim=64, hidden_layer_sizes=[32]):
    # Compute embedding for book_idxs.
    len_book_idxs = tf.keras.layers.Input([])
    book_idxs = tf.keras.layers.Input([None])
    # book_idxs % n_books is to prevent crashing if a book_idx in book_idxs is > n_books.
    book_idxs_emb = tf.keras.layers.Embedding(n_books, embed_dim)(book_idxs % n_books)
    book_idxs_emb = tf.math.divide(
        tf.keras.backend.sum(book_idxs_emb, axis=1), tf.expand_dims(len_book_idxs, 1)
    )
    # Compute embedding for book_idx.
    book_idx = tf.keras.layers.Input([])
    book_idx_emb = tf.keras.layers.Embedding(n_books, embed_dim)(book_idx)
    input_layer = tf.keras.layers.concatenate([book_idxs_emb, book_idx_emb], 1)
    # Build Multi Layer Perceptron on input layer.
    cur_layer = input_layer
    for size in hidden_layer_sizes:
        tf.keras.layers.Dense(size, activation=tf.nn.relu)(cur_layer)
    output_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(cur_layer)
    # Create and compile keras model.
    model = tf.keras.Model(
        inputs=[len_book_idxs, book_idxs, book_idx], outputs=[output_layer]
    )
    model.compile(
        "Adagrad",
        "binary_crossentropy",
        metrics=["accuracy", f1_batch, precision_batch, recall_batch],
    )
    return model

We use triples of (`book_idxs`, `book_idx`, `rating`) from our dataframes as training examples. In addition, we want to train the model to recognize when a user will not read a book. To create examples for that, we randomly sample a `book_id` not in `book_idxs` and use that with a `rating` of 0 as a _random negative_ example. We create one such _random negative_ example for every positive (`rating` 1) example in our dataframe so that positive and negative examples are roughly balanced.

In [12]:
# Generator to turn dataframe into examples.
def get_examples_generator(df):
    def generator():
        for book_idxs, book_idx, rating in zip(df.book_idxs, df.book_idx, df.rating):
            # Remove book_idx from book_idxs so the model can't just look it up.
            book_idxs = tuple(filter(lambda x: x != book_idx, book_idxs))
            yield {
                "len_book_idxs": len(book_idxs),
                "book_idxs": book_idxs,
                "book_idx": book_idx,
                "label": rating,
            }
            if rating == 1:
                # Generate a random negative book_id not in book_idxs.
                random_negative = np.random.randint(0, n_books)
                while random_negative in book_idxs:
                    random_negative = np.random.randint(0, n_books)
                yield {
                    "len_book_idxs": len(book_idxs),
                    "book_idxs": book_idxs,
                    "book_idx": random_negative,
                    "label": 0,
                }

    return generator


def get_data_tensors(df):
    # Use generator to get examples each epoch, along with shuffling and batching.
    padded_shapes = {
        "len_book_idxs": [],
        "book_idxs": [None],
        "book_idx": [],
        "label": [],
    }
    dataset = (
        tf.data.Dataset.from_generator(
            get_examples_generator(df), {k: tf.int64 for k in padded_shapes}
        )
        .shuffle(123)
        .repeat(None)
        .padded_batch(batch_size=256, padded_shapes=padded_shapes)
    )
    tensor_dict = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    return (
        tensor_dict["len_book_idxs"],
        tensor_dict["book_idxs"],
        tensor_dict["book_idx"],
        tensor_dict["label"],
    )

We now train the model on our combined training data (data labeled by LFs plus dev data).


In [13]:
from utils import get_n_epochs

model = get_model()

train_data_tensors = get_data_tensors(df_train_filtered)
valid_data_tensors = get_data_tensors(df_valid)
model.fit(
    train_data_tensors[:-1],
    train_data_tensors[-1],
    steps_per_epoch=300,
    validation_data=(valid_data_tensors[:-1], valid_data_tensors[-1]),
    validation_steps=40,
    epochs=get_n_epochs(),
    verbose=1,
)

W0813 19:24:18.100777 139779728627520 deprecation.py:506] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


W0813 19:24:18.262336 139779728627520 deprecation.py:506] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


W0813 19:24:18.381911 139779728627520 deprecation.py:323] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W0813 19:24:18.404990 139779728627520 deprecation.py:323] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


W0813 19:24:18.604613 139779728627520 deprecation.py:506] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/keras/optimizer_v2/adagrad.py:105: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

293/300 [============================>.] - ETA: 0s - loss: 0.5356 - acc: 0.7830 - f1_batch: 0.6614 - precision_batch: 0.8355 - recall_batch: 0.5513

295/300 [============================>.] - ETA: 0s - loss: 0.5358 - acc: 0.7830 - f1_batch: 0.6620 - precision_batch: 0.8354 - recall_batch: 0.5522

297/300 [============================>.] - ETA: 0s - loss: 0.5354 - acc: 0.7833 - f1_batch: 0.6619 - precision_batch: 0.8354 - recall_batch: 0.5521

299/300 [============================>.] - ETA: 0s - loss: 0.5352 - acc: 0.7832 - f1_batch: 0.6615 - precision_batch: 0.8353 - recall_batch: 0.5516

300/300 [==============================] - 13s 44ms/step - loss: 0.5352 - acc: 0.7832 - f1_batch: 0.6616 - precision_batch: 0.8352 - recall_batch: 0.5517 - val_loss: 0.6534 - val_acc: 0.6562 - val_f1_batch: 0.5095 - val_precision_batch: 0.5982 - val_recall_batch: 0.4551


Finally, we evaluate the model's predicted ratings on our test data.


In [14]:
test_data_tensors = get_data_tensors(df_test)
_ = model.evaluate(test_data_tensors[:-1], test_data_tensors[-1], steps=30)

 1/30 [>.............................] - ETA: 3s - loss: 0.7161 - acc: 0.5039 - f1_batch: 0.3981 - precision_batch: 0.3088 - recall_batch: 0.5600

 3/30 [==>...........................] - ETA: 1s - loss: 0.6778 - acc: 0.5990 - f1_batch: 0.4503 - precision_batch: 0.4902 - recall_batch: 0.4570

 5/30 [====>.........................] - ETA: 1s - loss: 0.6677 - acc: 0.6250 - f1_batch: 0.4679 - precision_batch: 0.5340 - recall_batch: 0.4427

 7/30 [======>.......................] - ETA: 0s - loss: 0.6614 - acc: 0.6311 - f1_batch: 0.4883 - precision_batch: 0.5469 - recall_batch: 0.4629

 9/30 [========>.....................] - ETA: 0s - loss: 0.6698 - acc: 0.6224 - f1_batch: 0.4764 - precision_batch: 0.5199 - recall_batch: 0.4589

...

23/30 [======================>.......] - ETA: 0s - loss: 0.6719 - acc: 0.6398 - f1_batch: 0.4954 - precision_batch: 0.5797 - recall_batch: 0.4454

25/30 [========================>.....] - ETA: 0s - loss: 0.6688 - acc: 0.6414 - f1_batch: 0.4907 - precision_batch: 0.5747 - recall_batch: 0.4400

27/30 [==========================>...] - ETA: 0s - loss: 0.6685 - acc: 0.6396 - f1_batch: 0.4894 - precision_batch: 0.5674 - recall_batch: 0.4433

29/30 [============================>.] - ETA: 0s - loss: 0.6681 - acc: 0.6393 - f1_batch: 0.4868 - precision_batch: 0.5706 - recall_batch: 0.4377

30/30 [==============================] - 1s 32ms/step - loss: 0.6685 - acc: 0.6388 - f1_batch: 0.4872 - precision_batch: 0.5687 - recall_batch: 0.4393


Our model has generalized quite well to our test set!
Note that we should additionally measure ranking metrics, like precision@10, before deploying to production.

## Summary

In this tutorial, we showed one way to use Snorkel for recommendations.
We used book metadata and review text to create LFs that estimate user ratings.
We used Snorkel's `LabelModel` to combine the outputs of those LFs.
Finally, we trained a model to predict whether a user will read and like a given book (and therefore what books should be recommended to the user) based only on what books the user has interacted with in the past.

Here we demonstrated one way to use Snorkel for training a recommender system.
Note, however, that this approach could easily be adapted to take advantage of additional information as it is available (e.g., user profile data, denser user ratings, and so on.)